In [1]:
import os, sys
from keras import backend as K

%matplotlib inline

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
homeDir = os.getcwd()
dataDir = homeDir + "/data/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"
test_path = dataDir + "test"
results_path = dataDir + "/results"

In [3]:
from utils import *
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

Fine tuning the previous best results of final layer tuning

In [4]:
vgg = Vgg16()

In [5]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=64

In [6]:
# get_batches function transforms images into arrays and the gets them in batches.
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [8]:
vgg.model.add(Dense(2, activation='softmax')) 
#Adding a new dense layer wiht only 2 outputs and softmax acitvation as it is the output layer

In [12]:
vgg.model.load_weights(model_path+'finetune_last_layer.h5')

In [10]:
# Using different optimizers 
opt = RMSprop(lr=0.00001)
vgg.model.compile(optimizer=opt,
                loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
23000/23000 [==============================] - 575s - loss: 0.0795 - acc: 0.9797 - val_loss: 0.0882 - val_acc: 0.9745


In [17]:
vgg.model.save_weights(model_path+'fineTuneLastLayerScore0_6.h5')

In [18]:
vgg.fit(batches, val_batches, nb_epoch=2)

Epoch 1/2
23000/23000 [==============================] - 575s - loss: 0.0838 - acc: 0.9783 - val_loss: 0.0874 - val_acc: 0.9740
Epoch 2/2
23000/23000 [==============================] - 575s - loss: 0.0841 - acc: 0.9785 - val_loss: 0.0943 - val_acc: 0.9735


In [32]:
vgg.model.save_weights(model_path+'fineTuneLastLayerScore0_6.h5')

In [33]:
vgg.fit(batches, val_batches, nb_epoch=3)

Epoch 1/3
23000/23000 [==============================] - 582s - loss: 0.0779 - acc: 0.9802 - val_loss: 0.0825 - val_acc: 0.9760
Epoch 2/3
23000/23000 [==============================] - 575s - loss: 0.0820 - acc: 0.9784 - val_loss: 0.0758 - val_acc: 0.9765
Epoch 3/3
23000/23000 [==============================] - 575s - loss: 0.0792 - acc: 0.9802 - val_loss: 0.0905 - val_acc: 0.9725


In [ ]:
vgg.model.save_weights(model_path+'fineTuneLastLayerScore0_6_after3.h5')

### Training dense layers wiht all the inputs

In [47]:
vgg.model.load_weights(model_path+'fineTuneLastLayerScore0_6.h5')

In [48]:
layers = vgg.model.layers

In [49]:
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]

In [50]:
for layer in layers[first_dense_idx:]: layer.trainable=True

In [51]:
# Using different optimizers 
opt = RMSprop(lr=0.0000001)
vgg.model.compile(optimizer=opt,
                loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
vgg.fit(batches, val_batches, nb_epoch=3)

Epoch 1/3
23000/23000 [==============================] - 634s - loss: 0.0780 - acc: 0.9798 - val_loss: 0.0858 - val_acc: 0.9760
Epoch 2/3
23000/23000 [==============================] - 634s - loss: 0.0754 - acc: 0.9809 - val_loss: 0.0864 - val_acc: 0.9755
Epoch 3/3
23000/23000 [==============================] - 633s - loss: 0.0772 - acc: 0.9804 - val_loss: 0.0867 - val_acc: 0.9755


In [53]:
vgg.model.save_weights(model_path+'fineTuneDenseLayerScore.h5')

In [59]:
opt = RMSprop(lr=0.000001)
vgg.model.compile(optimizer=opt,
                loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
23000/23000 [==============================] - 635s - loss: 0.0591 - acc: 0.9859 - val_loss: 0.1211 - val_acc: 0.9740


### Training the dense layers

In [9]:
vgg.model.load_weights(model_path + 'fineTuneLastLayerScore0_6.h5')

In [10]:
model = vgg.model
layers = model.layers
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]

In [11]:
batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [21]:
# Not needed everytime if we already have these features saved up.
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)

In [22]:
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [12]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [15]:
trn_features.shape

(23000, 512, 14, 14)

In [16]:
convLayers[-1].output_shape[1:]

(512, 14, 14)

In [17]:
def new_weights(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
opt = RMSprop(lr=0.0000001)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        Dropout(p),        
        Dense(4096, activation='relu'),
        Dropout(p),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2,0.5,p))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
fc_model = get_fc_model(0.5)

In [19]:
fc_model.fit(trn_features, trn_labels, nb_epoch=3, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 33s - loss: 0.0857 - acc: 0.9787 - val_loss: 0.0855 - val_acc: 0.9760
Epoch 2/3
23000/23000 [==============================] - 33s - loss: 0.0786 - acc: 0.9807 - val_loss: 0.0852 - val_acc: 0.9760
Epoch 3/3
23000/23000 [==============================] - 33s - loss: 0.0795 - acc: 0.9798 - val_loss: 0.0848 - val_acc: 0.9765


In [20]:
fc_model.fit(trn_features, trn_labels, nb_epoch=3, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 33s - loss: 0.0764 - acc: 0.9806 - val_loss: 0.0844 - val_acc: 0.9760
Epoch 2/3
23000/23000 [==============================] - 33s - loss: 0.0734 - acc: 0.9810 - val_loss: 0.0841 - val_acc: 0.9760
Epoch 3/3
23000/23000 [==============================] - 33s - loss: 0.0675 - acc: 0.9824 - val_loss: 0.0836 - val_acc: 0.9755


In [21]:
opt = RMSprop(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
fc_model.fit(trn_features, trn_labels, nb_epoch=1, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 33s - loss: 0.0758 - acc: 0.9839 - val_loss: 0.0838 - val_acc: 0.9815


In [23]:
for layer in convModel.layers: layer.trainable = False
# Look how easy it is to connect two models together!
convModel.add(fc_model)

In [24]:
convModel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
convModel.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
23000/23000 [==============================] - 635s - loss: 0.0096 - acc: 0.9984 - val_loss: 0.8106 - val_acc: 0.8970


In [26]:
convModel.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[1][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[1][0]            
___________________________________________________________________________________________

In [38]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)
preds = convModel.predict_generator(test_batches, test_batches.nb_sample)

Found 12500 images belonging to 1 classes.


In [48]:
preds.shape

(12500, 512, 14, 14)

In [39]:
filenames = test_batches.filenames
isdog = preds[:,1]
isdog = isdog.clip(min=0.005, max=0.995)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers_995.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

ValueError: all input arrays must have the same shape

### Predictions

In [61]:
results_path = dataDir + "/results"

In [62]:
batchnames, preds = vgg.test(test_path, batch_size = batch_size*2)

Found 12500 images belonging to 1 classes.


In [63]:
filenames = batchnames.filenames

In [64]:
isdog = preds[:,1]
isdog = isdog.clip(min=0.005, max=0.995)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data


In [65]:
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)

/home/ubuntu/KaggleCompetitions/cats_dogs_redux


/home/ubuntu/KaggleCompetitions/cats_dogs_redux/data/submission_ft_denseLayers1.csv